In [5]:
from regression_cascade import *
from sklearn.base import clone

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--list', default=None)
parser.add_argument('--train_test_path', default=None, help='If set, the computed feature vectors, labels and'
                                                            'responses will be cached.')
parser.add_argument('--output_path', default=None, type=str, help='The path where learned model is written.')
parser.add_argument('--subsample', default=1, type=int, help='If greater than 1, the training set will be'
                                                             'down-sampled.')
parser.add_argument('--step_size', default=10, type=int, help='Number of frame between two feature extractions.')
parser.add_argument('--train_ratio', default=0.99999, type=float)
parser.add_argument('--cv', default=3, type=int, help='Number of folds during cross-valication.')
parser.add_argument('--option', default=None, type=str, help='(Optional) The path to a file containing the '
                                                             'hyperparameters. If not provided, a grid search'
                                                             ' will be performed to obtain the best hyperparameter.')
args = parser.parse_args()

In [4]:
load_from_list = True
data_loaded = False
feature_train, label_train, responses_train = None, None, None
feature_test, label_test, responses_test = None, None, None
class_map = {}
train_file_path, test_file_path = None, None
# if args.train_test_path:
#     train_file_path = args.train_test_path + '/train.npy'
#     test_file_path = args.train_test_path + '/test.npy'
#     if os.path.exists(train_file_path) and os.path.exists(test_file_path) and os.path.exists(
#                     args.train_test_path + '/class_map.txt'):
#         print('Loading training set from ', train_file_path)
#         train_all = np.load(train_file_path)
#         print('Loading testing set from ', test_file_path)
#         test_all = np.load(test_file_path)
#         feature_train, label_train, responses_train = train_all[:, :-3], train_all[:, -3], train_all[:, -2:]
#         feature_test, label_test, responses_test = test_all[:, :-3], test_all[:, -3], test_all[:, -2:]
#         with open(args.train_test_path + '/class_map.txt') as f:
#             num_classes = int(f.readline().strip())
#             for i in range(num_classes):
#                 line = f.readline().strip().split()
#                 class_map[line[0]] = int(line[1])
#         load_from_list = False
#         data_loaded = True
list_path="../../data_publish_v2/list_small_train.txt"
if load_from_list:
    option = td.TrainingDataOption()
    option.sample_step_ = 10 # args.step_size
    feature_all, label_all, responses_all, class_map = load_datalist(path=list_path, option=option)
    responses_all = responses_all[:, [0, 2]]

    print('Data loaded. Total number of samples: ', feature_all.shape[0])

    for key, value in class_map.items():
        print('%d samples in %s(label %d)' % (len(label_all[label_all==value]), key, value))

    # Combine label and response to a single array to simplify the splitting process.
    target_temp = np.concatenate([label_all[:, None], responses_all], axis=1)
    feature_train, feature_test, target_train, target_test = train_test_split(feature_all, target_temp,
                                                                              train_size=0.99)
    print('Randomly splitting the dataset to %d/%d samples for training/testing.' %
          (feature_train.shape[0], feature_test.shape[0]))
    label_train, responses_train = target_train[:, 0], target_train[:, 1:]
    label_test, responses_test = target_test[:, 0], target_test[:, 1:]
    data_loaded = True
#     if args.train_test_path:
#         if not os.path.exists(args.train_test_path):
#             os.makedirs(args.train_test_path)
#         train_all = np.concatenate([feature_train, label_train[:, None], responses_train], axis=1)
#         test_all = np.concatenate([feature_test, label_test[:, None], responses_test], axis=1)
#         np.save(train_file_path, train_all)
#         np.save(test_file_path, test_all)
#         with open(args.train_test_path + '/class_map.txt', 'w') as f:
#             f.write('%d\n' % len(class_map))
#             for k, v in class_map.items():
#                 f.write('{:s} {:d}\n'.format(k, v))
#         print('Training/testing set written to ' + args.train_test_path)
if not data_loaded:
    raise ValueError('Both data list and train/test directory are invalid')

# if args.subsample > 1:
#     feature_train = feature_train[0:-1:args.subsample]
#     label_train = label_train[0:-1:args.subsample]
#     responses_train = responses_train[0:-1:args.subsample]

Loading dataset ../../data_publish_v2/zhicheng_leg1/processed/data.csv, type:  leg
Loading dataset ../../data_publish_v2/zhicheng_bag1/processed/data.csv, type:  bag
Loading dataset ../../data_publish_v2/zhicheng_handheld1/processed/data.csv, type:  handheld
Loading dataset ../../data_publish_v2/zhicheng_body1/processed/data.csv, type:  body 
Data loaded. Total number of samples:  10183
2750 samples in  leg(label 0)
2476 samples in  bag(label 1)
2452 samples in  handheld(label 2)
2505 samples in  body (label 3)
Randomly splitting the dataset to 10081/102 samples for training/testing.


/home/bixind/projects/strizh/ridi_imu/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [28]:
class CascadeTemplate:
    """
    The cascaded model consists of a classifier and num_classes * num_channels regressors.
    """
    def __init__(self, classifier, regressor, num_classes, num_channels, class_map):
        self.num_classes = num_classes
        self.num_channels = num_channels
        self.classifier = classifier
        self.regressors = [clone(regressor) for i in range(num_classes * num_channels)]
        self.class_map = class_map

    def train(self, train_feature, train_label, train_response):
        """
        Train the cascade model. It first trains the classifier with train_feature and train_label. Then the
        training samples are split into num_classes groups based on train_label. Regressors are trained for each
        group and each channel.
        
        :param train_feature: Nxd array containing N training feature vectors.
        :param train_label: Nx1 integer array containing N training labels.
        :param train_response: Nxc array where c equals to num_channels.
        :return: None
        """
        assert train_response.shape[1] == self.num_channels
        train_feature_cv = train_feature.astype(np.float32)
        if self.num_classes > 1:
            print('Training classifier')
            self.classifier.fit(train_feature, train_label)
            predicted_train = self.classifier.predict(train_feature_cv)
            error_svm = accuracy_score(train_label, predicted_train)
            print('Classifier trained. Training accuracy: %f' % error_svm)
        # Split the training sample based on ground truth label.
        for cls_name, cls in self.class_map.items():
            feature_in_class = train_feature_cv[train_label == cls, :]
            target_in_class = train_response[train_label == cls, :]
            # Skip models in 'transition' mode.
            if cls_name == ignore_class:
                continue
            for chn in range(self.num_channels):
                rid = cls * self.num_channels + chn
                print('Training regressor for class %d, channel %d' % (cls, chn))
                self.regressors[rid].fit(feature_in_class,
                                         target_in_class[:, chn].astype(np.float32))
                predicted = self.regressors[rid].predict(feature_in_class)
                print('Regressor for class %d  channel %d trained. Training error: %f(r2), %f(MSE)' %
                      (cls, chn, r2_score(predicted, target_in_class[:, chn]),
                       mean_squared_error(predicted, target_in_class[:, chn])))
        print('All done')

    def test(self, test_feature, true_label=None, true_responses=None):
        """
        Predict the label and responses of given samples.
        
        :param test_feature: Nxd array containing N testing feature vectors.
        :param true_label: Optional. Nx1 integer array containing N ground truth label.
        :param true_responses: Optional. Nxc array containing N ground truth responses.
        :return: predicted label and responses.
        """
        feature_cv = test_feature.astype(np.float32)
        labels = np.zeros(feature_cv.shape[0])
        if self.num_classes > 1:
            labels = self.classifier.predict(feature_cv)
            if true_label is not None:
                print('Classification accuracy: ', accuracy_score(true_label, labels))

        index_array = np.array([i for i in range(test_feature.shape[0])])
        reverse_index = [None for _ in self.class_map]
        predicted_class = [None for _ in self.class_map]
        for cls_name, cls in self.class_map.items():
            feature_in_class = feature_cv[labels == cls, :]
            predicted_in_class = np.zeros([feature_in_class.shape[0], self.num_channels])
            if feature_in_class.shape[0] > 0 and cls_name != ignore_class:
                for chn in range(self.num_channels):
                    rid = cls * self.num_channels + chn
                    predicted_in_class[:, chn] = self.regressors[rid].predict(feature_in_class)
            predicted_class[cls] = predicted_in_class
            reverse_index[cls] = index_array[labels == cls]
        if true_responses is not None:
            for cls_name, cls in self.class_map.items():
                if cls_name == ignore_class:
                    continue
                true_in_class = true_responses[labels == cls, :]
                if true_in_class.shape[0] == 0:
                    continue
                for chn in range(self.num_channels):
                    mse = mean_squared_error(true_in_class[:, chn], predicted_class[cls][:, chn])
                    print('Error for class %d, channel %d: %f(MSE)' % (cls, chn, mse))
        predicted_all = np.empty([test_feature.shape[0], self.num_channels])
        for cls_name, cls in self.class_map.items():
            predicted_all[reverse_index[cls], :] = predicted_class[cls]

        if true_responses is not None:
            for chn in range(self.num_channels):
                mse = mean_squared_error(true_responses[:, chn], predicted_all[:, chn])
                print('Overall regression error for channel %d: %f(MSE)' % (chn, mse))
        return labels, predicted_all


In [7]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

In [29]:
clf = KNeighborsClassifier()
clf.fit(feature_train, label_train.astype(np.int32))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [42]:
%%time
model = CascadeTemplate(KNeighborsClassifier(n_neighbors=40), KNeighborsRegressor(n_neighbors=40),
                     len(class_map), responses_train.shape[1], class_map)
print('Sample used for training: ', feature_train.shape[0])
model.train(feature_train, label_train.astype(np.int32), responses_train)

Sample used for training:  10081
Training classifier
Classifier trained. Training accuracy: 0.920246
Training regressor for class 0, channel 0
Regressor for class 0  channel 0 trained. Training error: 0.860663(r2), 0.005970(MSE)
Training regressor for class 0, channel 1
Regressor for class 0  channel 1 trained. Training error: 0.792729(r2), 0.020685(MSE)
Training regressor for class 1, channel 0
Regressor for class 1  channel 0 trained. Training error: 0.529064(r2), 0.029316(MSE)
Training regressor for class 1, channel 1
Regressor for class 1  channel 1 trained. Training error: 0.117832(r2), 0.015405(MSE)
Training regressor for class 2, channel 0
Regressor for class 2  channel 0 trained. Training error: -20.260236(r2), 0.066690(MSE)
Training regressor for class 2, channel 1
Regressor for class 2  channel 1 trained. Training error: -0.051769(r2), 0.040288(MSE)
Training regressor for class 3, channel 0
Regressor for class 3  channel 0 trained. Training error: -1.685126(r2), 0.030228(MSE)

In [35]:
def read_list(list_path):
    option = td.TrainingDataOption()
    option.sample_step_ = 10 # args.step_size
    feature_all, label_all, responses_all, class_map = load_datalist(path=list_path, option=option)
    responses_all = responses_all[:, [0, 2]]

    print('Data loaded. Total number of samples: ', feature_all.shape[0])

    for key, value in class_map.items():
        print('%d samples in %s(label %d)' % (len(label_all[label_all==value]), key, value))

    # Combine label and response to a single array to simplify the splitting process.
    target_temp = np.concatenate([label_all[:, None], responses_all], axis=1)
    
    
    label, responses = target_temp[:, 0], target_temp[:, 1:]
    return feature_all, label.astype(np.int32), responses

In [36]:
feature_val, label_val, responses_val = read_list("../../data_publish_v2/list_small_test.txt")

Loading dataset ../../data_publish_v2/ruixuan_leg1/processed/data.csv, type:  leg
Loading dataset ../../data_publish_v2/ruixuan_bag1/processed/data.csv, type:  bag
Loading dataset ../../data_publish_v2/ruixuan_handheld1/processed/data.csv, type:  handheld
Loading dataset ../../data_publish_v2/ruixuan_body2/processed/data.csv, type:  body 
Data loaded. Total number of samples:  9317
2802 samples in  leg(label 0)
2547 samples in  bag(label 1)
2694 samples in  handheld(label 2)
1274 samples in  body (label 3)


In [40]:
responses_val.mean(axis=0)

array([-0.15656441, -0.62959046])

In [43]:
%%time
res_labels, res_predicted = model.test(feature_val, label_val, responses_val)

Classification accuracy:  0.4132231404958678
Error for class 0, channel 0: 0.070734(MSE)
Error for class 0, channel 1: 0.123183(MSE)
Error for class 1, channel 0: 0.096159(MSE)
Error for class 1, channel 1: 0.015070(MSE)
Error for class 2, channel 0: 0.482002(MSE)
Error for class 2, channel 1: 0.786707(MSE)
Error for class 3, channel 0: 0.101943(MSE)
Error for class 3, channel 1: 0.074792(MSE)
Overall regression error for channel 0: 0.417886(MSE)
Overall regression error for channel 1: 0.675967(MSE)
CPU times: user 2.73 s, sys: 100 ms, total: 2.83 s
Wall time: 2.83 s


In [47]:
def stuff():
    svm_option = SVMOption()
    svm_option.svm_type = cv2.ml.SVM_C_SVC
    svm_option.kernel_type = cv2.ml.SVM_RBF
    svm_option.C = 10.0
    svm_option.gamma = 1. / feature_train.shape[1]

    svr_options = []
    num_classes = max(label_train) + 1
    assert num_classes == len(class_map)
    num_channels = responses_train.shape[1]
    for cls_name, cls in class_map.items():
        for chn in range(num_channels):
            svr_option = SVMOption()
            svr_option.svm_type = cv2.ml.SVM_EPS_SVR
            svr_option.kernel_type = cv2.ml.SVM_RBF
            svr_option.gamma = 1. / feature_train.shape[1]
            if cls_name is not ignore_class:
                svr_option.C = 1
                svr_option.e = 0.01
            svr_options.append(svr_option)
    print('All done')
    return SVRCascadeOption(num_classes, num_channels, svm_option, svr_options)
option = stuff()

All done


In [48]:
bmodel = SVRCascade(option, class_map)
print('Sample used for training: ', feature_train.shape[0])
bmodel.train(feature_train, label_train.astype(np.int32), responses_train)

Sample used for training:  10081
Training classifier
Classifier trained. Training accuracy: 0.991271
Training regressor for class 0, channel 0
Regressor for class 0  channel 0 trained. Training error: 0.997724(r2), 0.000102(MSE)
Training regressor for class 0, channel 1
Regressor for class 0  channel 1 trained. Training error: 0.998848(r2), 0.000114(MSE)
Training regressor for class 1, channel 0
Regressor for class 1  channel 0 trained. Training error: 0.998039(r2), 0.000127(MSE)
Training regressor for class 1, channel 1
Regressor for class 1  channel 1 trained. Training error: 0.996128(r2), 0.000102(MSE)
Training regressor for class 2, channel 0
Regressor for class 2  channel 0 trained. Training error: 0.342953(r2), 0.020653(MSE)
Training regressor for class 2, channel 1
Regressor for class 2  channel 1 trained. Training error: 0.422518(r2), 0.022804(MSE)
Training regressor for class 3, channel 0
Regressor for class 3  channel 0 trained. Training error: 0.997741(r2), 0.000101(MSE)
Tra

In [50]:
%%time
res_labels, res_predicted = bmodel.test(feature_val, label_val, responses_val)

Classification accuracy:  0.9541697971450037
Error for class 0, channel 0: 0.045992(MSE)
Error for class 0, channel 1: 0.037454(MSE)
Error for class 1, channel 0: 0.017859(MSE)
Error for class 1, channel 1: 0.019767(MSE)
Error for class 2, channel 0: 0.042062(MSE)
Error for class 2, channel 1: 0.036382(MSE)
Error for class 3, channel 0: 0.044325(MSE)
Error for class 3, channel 1: 0.016320(MSE)
Overall regression error for channel 0: 0.037335(MSE)
Overall regression error for channel 1: 0.029780(MSE)
CPU times: user 3min 45s, sys: 113 ms, total: 3min 45s
Wall time: 28.9 s
